# Imports

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications import EfficientNetB0

import os
import numpy as np


# Augmentations, Preprocessing and Preparation

In [ ]:
data_dir = 'data/RecyclableAndHouseholdWasteClassification/images'

image_size = (256, 256) # also try training using 256x256 images
batch_size = 32 # change this to 16, 32, 64, or 128 and compare the results
seed = 1337 # for repeatability in train/validation split

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0, # normalize pixels
    validation_split=0.2, # 80/20 percent split
    rotation_range = 20, # rotate images 20 degrees CW or CCW
    width_shift_range = 0.1, # shift images up or down 10 %
    height_shift_range = 0.1, # shift images right or left 10%
    shear_range = 0.1, #shears (distorts) images
    zoom_range = 0.15, # zoom in or out
    horizontal_flip = True,
    brightness_range = [0.8, 0.12], # mimics real-world brightness inconsistency 
    fill_mode = 'nearest' # fill in new pixels during augumentations 
)

valid_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    validation_split = 0.2
    
)

# load train/valid data

train_data = datagen.flow_from_directory(data_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical', subset='training', shuffle = True, seed = seed)

valid_data = datagen.flow_from_directory(data_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical', subset='validation', shuffle = True, seed = seed)

num_classes = len(train_data.class_indices)

Found 12000 images belonging to 30 classes.
Found 3000 images belonging to 30 classes.


# Build the Model off a EfficientNetB0 model

In [ ]:
base_model = EfficientNetB0(
    include_top = False, 
    weights = 'imagenet',
    input_shape=(image_size[0], image_size[1], 3) #keep size always consistent
)

base_model.trainable = False

inputs = layers.Input(shape=(image_size[0], image_size[1], 3))

x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x) # drop 40% of neurons; avoid overfitting

outputs = layers.Dense(num_classes, activation='softmax')(x)
model = models.Model(inputs=inputs, outputs=outputs, name="efficientnetb0_transfer")
model.summary()

# Compile the Model

# Fit the Model

In [10]:
# Train the model
history = model.fit(
    train_data,
    epochs=10,  # play with this number to maximize accuracy
    validation_data=valid_data
)

Epoch 1/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 170s 451ms/step - accuracy: 0.1508 - loss: 4.4899 - val_accuracy: 0.4187 - val_loss: 2.1075
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 183s 487ms/step - accuracy: 0.6150 - loss: 1.4932 - val_accuracy: 0.6103 - val_loss: 1.5791
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 190s 506ms/step - accuracy: 0.8770 - loss: 0.5025 - val_accuracy: 0.6420 - val_loss: 1.6903
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 191s 510ms/step - accuracy: 0.9441 - loss: 0.2489 - val_accuracy: 0.6633 - val_loss: 1.5205
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 174s 464ms/step - accuracy: 0.9607 - loss: 0.1618 - val_accuracy: 0.6763 - val_loss: 1.5118
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 181s 483ms/step - accuracy: 0.9659 - loss: 0.1275 - val_accuracy: 0.6813 - val_loss: 1.5385
Epoch 7/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 137s 365ms/step - accuracy: 0.9617 - loss: 0.1234 - val_accuracy: 0.6727 - val_loss: 1.4568
Epoch 8/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 143s 382ms/step - accuracy: 0.9668 -

# Validation for accuracy
Note: check `validation_split=0.05` in "Preprocessing and Preparation." Use `validation_split=0.2` to get an accuracy score that better reflects the model's performance.  

In [11]:
val_loss, val_acc = model.evaluate(valid_data)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_acc}')

train_loss, train_acc = model.evaluate(train_data)
print(f'Training Loss: {train_loss}, Training Accuracy: {train_acc}')

94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 109ms/step - accuracy: 0.6541 - loss: 1.6623
Validation Loss: 1.7113932371139526, Validation Accuracy: 0.656000018119812
375/375 ━━━━━━━━━━━━━━━━━━━━ 39s 104ms/step - accuracy: 0.9707 - loss: 0.1036
Training Loss: 0.09901190549135208, Training Accuracy: 0.9704166650772095


# Save the model

In [14]:
model.save('model.keras')

# Use the Model to Make a Prediction for a user uploaded Image

In [25]:
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

# Load the image
uploaded_img_path = 'test_imgs/img4.jpeg'  # Replace with the actual path of the uploaded image
uploaded_img = load_img(uploaded_img_path, target_size=image_size)

# Convert the image to an array and normalize it
uploaded_img_array = img_to_array(uploaded_img) / 255.0
uploaded_img_array = np.expand_dims(uploaded_img_array, axis=0)  # Add batch dimension

# Make a prediction
uploaded_prediction = model.predict(uploaded_img_array)

# Decode the prediction
predicted_class = np.argmax(uploaded_prediction, axis=1)
class_labels = {v: k for k, v in train_data.class_indices.items()}  # Reverse the class indices dictionary
predicted_label = class_labels[predicted_class[0]]

print(f'Predicted class: {predicted_label}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Predicted class: plastic_food_containers


In [26]:
len(train_data.class_indices)

30